In [185]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [186]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [187]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [188]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [189]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [190]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [191]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [192]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [193]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [194]:
with open('label_encoder_gender.pickle', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pickle', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)    

In [195]:
X = data.drop('Exited', axis=1)
y = data['Exited']

In [196]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [197]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(X_test)

In [198]:
with open('scaler.pickle', 'wb') as file:
    pickle.dump(scaler,file)

ANN Classification

In [199]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [200]:
model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

c:\Users\dell\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [201]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [202]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate = 0.01)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [203]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [204]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs = 100,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8363 - loss: 0.3989 - val_accuracy: 0.8035 - val_loss: 2462.5271
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8554 - loss: 0.3557 - val_accuracy: 0.8035 - val_loss: 5308.2227
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8570 - loss: 0.3502 - val_accuracy: 0.8035 - val_loss: 3378.4343
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8580 - loss: 0.3436 - val_accuracy: 0.4810 - val_loss: 8172.6426
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8611 - loss: 0.3403 - val_accuracy: 0.5820 - val_loss: 10266.7090
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8593 - loss: 0.3371 - val_accuracy: 0.5945 - val_loss: 7320.7842


In [205]:
model.save('model.h5')

In [206]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [207]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 16576), started 6:34:20 ago. (Use '!kill 16576' to kill it.)

In [208]:
from tensorflow.keras.models import load_model
import pickle

In [209]:
model = load_model('model.h5')

In [210]:
with open('onehot_encoder_geo.pickle','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pickle','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pickle','rb') as file:
    scaler = pickle.load(file)       

In [211]:
import pandas as pd

In [212]:
input_data = {
    'CreditScore': [600],
    'Geography': ['France'],
    'Gender': ['Male'],
    'Age': [40],
    'Tenure': [30],
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

input_df = pd.DataFrame(input_data)

In [213]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,30,60000,2,1,1,50000


In [214]:
geo_encoded = label_encoder_geo.transform(input_df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [215]:
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,30,60000,2,1,1,50000,1.0,0.0,0.0


In [216]:
input_scaled = scaler.transform(input_df)

In [217]:
prediction = model.predict(input_scaled)

print(f"Prediction Probaility: {prediction[0][0]:.4f}")

if prediction[0][0] > 0.5:
    print('The customer is likely to churn.')

else:
    print("The customer is not likely to churn.")    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
Prediction Probaility: 0.0156
The customer is not likely to churn.
